# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [3]:
import random, math
import pandas as pd
import time
import os

In [4]:
def save_instance(dataset, instance_name, n, aristas):
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        print(n, len(aristas), file=f) #escribo la cantidad de nodos seguido de la cantidad de aristas
        for arista in aristas: #para todas las aristas
            print(arista[0], file=f, end= " ") #escribo el primer nodo
            print(arista[1], file=f, end= " ") #el segundo
            print(arista[2], file=f, end =" ") # el peso y salto de linea
            
def save_index(dataset, instances):
    with open(F"instancias/{dataset}/instances.txt", "w") as f:
        for instance in instances: 
            print(instance, file=f)

In [3]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
              S.append([i, j, random.randint(0, 100)]) #la guardo
    save_instance("grafo-propio", F"G-{n}", n, S)
    filas_indice.append(["grafo-propio", F"G-{n}", n, aristas, F"instancias/grafo-propio/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/grafo-propio/indice.csv", index=False, header=True)    

# Peor caso AGM

In [4]:
filas_agm = []
for n in range(3, 103):
    aristas = n*(n-1)/2
    S = []
    for j in range(2,n+1):
        S.append([1, j, 1]) #quiero que el AGM sea una estrella centrada en V1
    for i in range(2, n+1): #el resto
        if i+1<n+1:
            S.append([i,i+1, random.randint(150, 200)]) #la arista hacia el proximo en dfs es MUY costosa
        for j in range(i+2, n+1):
            S.append([i,j, random.randint(20,100)]) #el resto es malo, pero no tanto como las que elije el dfs
    save_instance("peor-caso-agm", F"AGM-{n}",n,S)
    filas_agm.append(["peor-caso-agm",F"AGM-{n}",n,aristas,F"instancias/peor-caso-agm/AGM-{n}.txt"])
pd.DataFrame(filas_agm, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-agm/indice.csv", index=False, header=True)

# Cargamos los grafos para los cuales conocemos los resultados optimos

In [6]:
filas_indice=[]
archivos = os.listdir("instancias/descargas/data")
for instancia in archivos:
    n = ""
    for letra in instancia:
        if letra.isnumeric():
            n = n+letra
    n = int(n)
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    nombre = instancia.split(".")[0]
    soluciones = open("instancias/descargas/pesos_optimos.txt", "r")
    encontre = False
    peso_opt = 0
    print(nombre)
    while(not encontre):
        linea = soluciones.readline().split(":")
        if nombre == linea[0]:
            encontre = True
            peso_opt = int(linea[1])
        if "FIN" == linea[0]:
            break
    if peso_opt == 0:
            print("falta el optimo del caso " + nombre)
            time.sleep(5)
    filas_indice.append(["sol_optimas", nombre, n, aristas, F"instancias/descargas/data/{nombre}.tsp", peso_opt])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo", "peso_optimo"]).to_csv("instancias/descargas/indice.csv", index=False, header=True)
    


u1432
gr666
rl1304
pr107
rat99
d2103
kroA200
rl1323
p654
u574
pr152
bier127
pcb1173
bays29
u1060
pr76
fl1577
kroB100
u2152
ch150
pr264
gr202
att532
dantzig42
kroB150
eil76
ts225
kroB200
pr144
u724
pa561
linhp318
u2319
pr2392
dsj1000
kroA150
kroA100
burma14
u159
d1655
gr229
gr137
kroC100
ch130
pr136
a280
eil51
lin105
rat195
gil262
d657
bayg29
gr431
fl417
pr299
pr1002
tsp225
nrw1379
ulysses16
vm1748
eil101
vm1084
rl1889
u1817
pr124
berlin52
d1291
d198
pr439
st70
pcb442
gr96
rat783
gr120
pr226
rd400
kroD100
fl1400
ali535
lin318
rat575
kroE100
att48
ulysses22
d493
rd100


# Peor Caso GA 

In [5]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    orden = [] #inicializo el vector que tendrá el orden del ciclo malo para GA
    
    for i in range (1,n+1):
        orden.append(i) #lleno con los vértices
    elementos_orden = orden #copio para hacer un shuffle
    random.shuffle(elementos_orden) #creo un orden random para los vértices y hacer un ciclo malo para GA
    print(elementos_orden)
    for i in range(0,n):
        orden[elementos_orden[i]-1] = i+1 #escribo en orden para que el indice sea el vértice y el elemento sea el orden en el ciclo 
    
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
            if orden[i-1] - orden[j-1] == 1 | orden[j-1] - orden[i-1] == 1: #si los vértices son consecutivos segun el orden
                S.append([i,j, random.randint(1,49)]) #guardo una arista pequeña
            elif (orden[i-1] == 1 & orden[j-1] == n) | (orden[j-1] == 1 & orden[i-1] == n): #si estamos hablando de la utima conexion del ciclo
                S.append([i,j, random.randint(n*100, n*500)]) #guardo una arista inutilizable
            else: #si los vértices no son consecutivos en orden
                S.append([i, j, random.randint(50, 100)]) #uso una arista de peso medio
    save_instance("peor-caso-ga", F"G-{n}", n, S)
    filas_indice.append(["peor-caso-ga", F"G-{n}", n, aristas, F"instancias/peor-caso-ga/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-ga/indice.csv", index=False, header=True)    

[3, 1, 2]
[4, 2, 3, 1]
[2, 3, 1, 4, 5]
[5, 3, 1, 2, 6, 4]
[2, 4, 5, 7, 1, 3, 6]
[8, 6, 7, 5, 3, 1, 4, 2]
[5, 6, 4, 8, 1, 3, 2, 7, 9]
[7, 6, 3, 10, 1, 9, 4, 8, 2, 5]
[7, 6, 5, 3, 4, 2, 10, 1, 9, 8, 11]
[6, 7, 3, 8, 9, 4, 11, 2, 5, 1, 12, 10]
[13, 8, 7, 2, 10, 11, 6, 3, 5, 1, 9, 4, 12]
[14, 9, 2, 10, 12, 11, 8, 5, 13, 1, 6, 4, 7, 3]
[6, 13, 7, 15, 12, 2, 14, 10, 9, 5, 11, 8, 3, 4, 1]
[12, 3, 8, 4, 10, 1, 9, 16, 6, 7, 15, 11, 5, 13, 2, 14]
[14, 6, 3, 7, 16, 12, 15, 13, 17, 4, 2, 11, 8, 10, 9, 1, 5]
[17, 13, 11, 3, 16, 5, 6, 7, 12, 8, 2, 10, 9, 1, 18, 15, 4, 14]
[17, 5, 19, 9, 18, 3, 10, 14, 4, 8, 13, 2, 6, 15, 16, 12, 7, 1, 11]
[9, 12, 11, 14, 19, 15, 2, 6, 3, 10, 18, 7, 4, 8, 13, 5, 16, 1, 17, 20]
[7, 16, 20, 19, 14, 2, 18, 10, 4, 21, 12, 3, 9, 11, 17, 15, 6, 1, 13, 8, 5]
[15, 9, 22, 16, 21, 7, 18, 17, 8, 1, 4, 19, 10, 6, 5, 12, 11, 20, 2, 3, 14, 13]
[15, 8, 13, 10, 14, 18, 23, 9, 21, 16, 19, 5, 3, 4, 17, 12, 20, 7, 11, 1, 22, 2, 6]
[22, 18, 9, 11, 6, 14, 2, 1, 13, 24, 8, 23, 15, 10, 19,

[6, 26, 60, 24, 63, 37, 7, 71, 74, 69, 42, 16, 59, 50, 53, 25, 17, 73, 65, 20, 34, 30, 57, 35, 48, 22, 41, 28, 68, 12, 10, 21, 8, 15, 49, 58, 72, 47, 39, 75, 1, 38, 3, 55, 23, 45, 32, 4, 54, 46, 11, 61, 18, 62, 51, 70, 36, 5, 67, 33, 66, 40, 14, 56, 29, 19, 27, 31, 43, 44, 9, 52, 2, 64, 13]
[60, 55, 66, 35, 49, 46, 75, 69, 37, 39, 56, 67, 42, 34, 27, 1, 51, 63, 26, 57, 71, 44, 16, 38, 30, 13, 12, 31, 17, 28, 50, 22, 32, 76, 73, 59, 53, 64, 61, 6, 48, 72, 45, 10, 62, 3, 47, 58, 21, 40, 18, 36, 70, 41, 11, 43, 74, 19, 24, 54, 29, 15, 7, 68, 52, 2, 14, 25, 5, 23, 20, 4, 8, 65, 33, 9]
[67, 66, 35, 8, 64, 53, 44, 46, 14, 63, 60, 55, 26, 12, 17, 72, 4, 24, 61, 3, 10, 30, 27, 28, 36, 9, 6, 29, 70, 77, 49, 5, 74, 57, 62, 18, 15, 42, 33, 76, 69, 59, 45, 48, 31, 21, 23, 11, 22, 50, 47, 73, 65, 51, 34, 39, 52, 13, 7, 2, 37, 43, 54, 40, 41, 75, 16, 38, 32, 25, 58, 56, 1, 20, 68, 19, 71]
[48, 60, 9, 61, 4, 3, 63, 69, 59, 67, 75, 36, 47, 77, 20, 26, 39, 65, 34, 7, 2, 11, 62, 40, 64, 56, 51, 54, 22, 

In [ ]:
#agrego comentario para pushear


# Eulerianos

In [3]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    base = random.randint(200,500) #el int que usamos de bas epara los pesos
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
              S.append([i, j, base + random.randint(0, base-1)]) #la guardo, con un peso entre la base y 2*base-1, asi siempre es más caro pasar por dos aristas en vez de una
    save_instance("grafo-euleriano", F"E-{n}", n, S)
    filas_indice.append(["grafo-euleriano", F"E-{n}", n, aristas, F"instancias/grafo-euleriano/E-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/grafo-euleriano/indice.csv", index=False, header=True)    

In [11]:
filas_indice = []
for n in range(3, 103):
    aristas = n*(n-1)/2
    s=[]
    for i in range(1, n+1):
        for j in range(i+1, n+1):
            if j==i+1:
                s.append([i, j, 1])
            elif i==1 and j==n:
                s.append([i, j, 1000000])
            else:    
                s.append([i, j, 2])
    save_instance("peor-caso-ga", F"G-{n}", n, s)
    filas_indice.append(["peor-caso-ga", F"G-{n}", n, aristas, F"instancias/peor-caso-ga/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-ga/indice2.csv", index=False, header=True)       